In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf, pandas as pd

In [2]:
df = cudf.read_parquet('../../data/train_data/test.parquet')
mn = df.ts.min()
#df['hour'] = df.ts % (60*60*24)
#df['day'] = df.ts % (60*60*24*7)
#df['aid2'] = df.aid
print( df.shape )
df.head()

(7683577, 4)


,session,aid,ts,type
0,11098528,11830,1661119200,0
1,11098529,1105029,1661119200,0
2,11098530,264500,1661119200,0
3,11098530,264500,1661119288,0
4,11098530,409236,1661119369,0


In [3]:
user_features = df.groupby('session').agg({'ts':'max'})
user_features.columns = ['ts_max7']

In [4]:
user_features.head()

,ts_max7
session,
12150265,1661636717
12720718,1661685970
12829669,1661707886
11939634,1661588668
11959102,1661415946


In [5]:
user_features2 = df.groupby('session').agg({'ts':'min'})
user_features2.columns = ['ts_min7']

In [6]:
user_features2.head()

,ts_min7
session,
12150265,1661461408
12720718,1661685970
12829669,1661707886
11939634,1661404870
11959102,1661412885


In [7]:
user_features = cudf.concat([user_features,user_features2],axis=1)

In [8]:
user_features['ts_diff7'] = user_features.ts_max7 - user_features.ts_min7

In [9]:
for c in ['ts_max7','ts_min7']:
    user_features[c] -= mn

In [10]:
user_features.head()

,ts_max7,ts_min7,ts_diff7
session,,,
11098528,0,0,0
11098529,0,0,0
11098530,1332,0,1332
11098531,546,0,546
11098532,796,1,795


In [11]:
user_features = user_features.astype('int32')

In [12]:
user_features.to_parquet('../../data/item_user_features/user7.pqt')